In [59]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType
# from pyspark.ml.stat import Summarizer
from pyspark.sql.functions import col, avg, regexp_extract


In [3]:
!which python
!echo $PYSPARK_PYTHON 
!echo $PYSPARK_DRIVER_PYTHON

/usr/local/anaconda3/envs/spark/bin/python




In [4]:
spark = SparkSession.builder.appName("dataload").getOrCreate()

print(f"Spark Instance: {spark}")
rdd = spark.sparkContext.parallelize([1,2,3,4])
print(f"RDD count: {rdd.count()}")

Spark Instance: <pyspark.sql.session.SparkSession object at 0x7fb6a6c80070>
RDD count: 4


In [5]:

print("load a csv:")
df = spark.read.csv('../sample_data/ramen-ratings.csv', header=True, inferSchema=True)

load a csv:


In [6]:
df.printSchema()

root
 |-- Review #: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Variety: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Stars: string (nullable = true)
 |-- Top Ten: string (nullable = true)



In [46]:
df.show(5, truncate=False)

+--------+--------------+-----------------------------------------------------------+-----+-------+-----+-------+
|Review #|Brand         |Variety                                                    |Style|Country|Stars|Top Ten|
+--------+--------------+-----------------------------------------------------------+-----+-------+-----+-------+
|2580    |New Touch     |T's Restaurant Tantanmen                                   |Cup  |Japan  |3    |null   |
|2579    |Just Way      |Noodles Spicy Hot Sesame Spicy Hot Sesame Guan-miao Noodles|Pack |Taiwan |1    |null   |
|2578    |Nissin        |Cup Noodles Chicken Vegetable                              |Cup  |USA    |2    |null   |
|2577    |Wei Lih       |GGE Ramen Snack Tomato Flavor                              |Pack |Taiwan |2    |null   |
|2576    |Ching's Secret|Singapore Curry                                            |Pack |India  |3    |null   |
+--------+--------------+-----------------------------------------------------------+---

In [8]:
print(f"Row count: {df.count()}")

Row count: 2584


In [10]:
print("Check the df schema after changing the data type")
df = df.withColumn('Stars', F.col("Stars").cast(IntegerType()))
df.printSchema()

Check the df schema after changing the data type
root
 |-- Review #: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Variety: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Stars: integer (nullable = true)
 |-- Top Ten: string (nullable = true)



In [11]:
df.describe()

DataFrame[summary: string, Review #: string, Brand: string, Variety: string, Style: string, Country: string, Stars: string, Top Ten: string]

In [12]:
df.summary()

DataFrame[summary: string, Review #: string, Brand: string, Variety: string, Style: string, Country: string, Stars: string, Top Ten: string]

In [29]:
print("The Avg Star Rating across all rows")
df.agg(avg(col('Stars'))).show()

+-----------------+
|       avg(Stars)|
+-----------------+
|3.347303065580132|
+-----------------+



In [30]:
print("The Avg Star Rating by Country")
df.groupby('Country').mean().show()

The Avg Star Rating by Country
+-------------+------------------+
|      Country|        avg(Stars)|
+-------------+------------------+
|       Sweden|               3.0|
|  Philippines|2.9574468085106385|
|    Singapore|3.8623853211009176|
|     Malaysia|3.8838709677419354|
|         Fiji|              3.75|
|      Germany|3.2222222222222223|
|     Cambodia|               4.0|
|       Taiwan|3.3705357142857144|
|         null|              null|
|      Finland|               3.0|
|        Ghana|               3.0|
|      Myanmar|               3.5|
|        India|               3.0|
|        China| 3.094674556213018|
|United States|               3.0|
|      Nigeria|               1.0|
|   Bangladesh|3.5714285714285716|
|     Thailand| 3.068062827225131|
|    Hong Kong| 3.562043795620438|
|          USA| 3.136222910216718|
+-------------+------------------+
only showing top 20 rows



In [32]:
print("The Avg Star Rating by Type")
df.groupby('Style').mean().show()

The Avg Star Rating by Type
+-----+------------------+
|Style|        avg(Stars)|
+-----+------------------+
| Bowl|3.3534303534303533|
|  Bar|               5.0|
|  Box| 4.166666666666667|
| null|               3.0|
| Pack| 3.393324607329843|
|  Cup|3.1977777777777776|
| Tray| 3.240740740740741|
|  Can|               3.0|
+-----+------------------+



In [99]:
print("Filter a column using Text")

df_2016 = df.where(F.col("Top Ten").rlike("2016"))
df_2016.show(100, truncate=False)

Filter a column using Text
+--------+-------------+-----------------------------------------------+-----+---------+-----+--------+
|Review #|Brand        |Variety                                        |Style|Country  |Stars|Top Ten |
+--------+-------------+-----------------------------------------------+-----+---------+-----+--------+
|1964    |MAMA         |Instant Noodles Coconut Milk Flavour           |Pack |Myanmar  |5    |2016 #10|
|1947    |Prima Taste  |Singapore Laksa Wholegrain La Mian             |Pack |Singapore|5    |2016 #1 |
|1925    |Prima        |Juzz's Mee Creamy Chicken Flavour              |Pack |Singapore|5    |2016 #8 |
|1907    |Prima Taste  |Singapore Curry Wholegrain La Mian             |Pack |Singapore|5    |2016 #5 |
|1828    |Tseng Noodles|Scallion With Sichuan Pepper  Flavor           |Pack |Taiwan   |5    |2016 #9 |
|1689    |Wugudaochang |Tomato Beef Brisket Flavor Purple Potato Noodle|Pack |China    |5    |2016 #7 |
+--------+-------------+-------------

In [58]:
print("Filter a column using Text and regex")

df_2016 = df.where(F.col("Top Ten").rlike("(#1)"))
df_2016.show(100, truncate=False)

Filter a column using Text and regex
+--------+----------------+----------------------------------------------------------+-----+---------+-----+--------+
|Review #|Brand           |Variety                                                   |Style|Country  |Stars|Top Ten |
+--------+----------------+----------------------------------------------------------+-----+---------+-----+--------+
|1964    |MAMA            |Instant Noodles Coconut Milk Flavour                      |Pack |Myanmar  |5    |2016 #10|
|1947    |Prima Taste     |Singapore Laksa Wholegrain La Mian                        |Pack |Singapore|5    |2016 #1 |
|1638    |A-Sha Dry Noodle|Veggie Noodle Tomato Noodle With Vine Ripened Tomato Sauce|Pack |Taiwan   |5    |2015 #10|
|1500    |MyKuali         |Penang Red Tom Yum Goong Noodle                           |Pack |Malaysia |5    |2015 #1 |
|1471    |Mama            |Instant Noodles Shrimp Creamy Tom Yum Flavour Jumbo Pack  |Pack |Thailand |5    |2013 #10|
|1302    |Mama     

In [98]:
print("filter by multiple conditions")
# result= df.where(F.col('Top Ten').rlike("#1"))
df.filter((F.col('Review #')=='1964') | (F.col('Review #')=='105') ).show()

filter by multiple conditions
+--------+-------+--------------------+-----+---------+-----+--------+
|Review #|  Brand|             Variety|Style|  Country|Stars| Top Ten|
+--------+-------+--------------------+-----+---------+-----+--------+
|    1964|   MAMA|Instant Noodles C...| Pack|  Myanmar|    5|2016 #10|
|     105|Indomie|Special Fried Cur...| Pack|Indonesia|    5| 2012 #1|
+--------+-------+--------------------+-----+---------+-----+--------+



In [100]:
print("WIP")
print("Extract some text from a field")
# df_rank = df.


# result = df.withColumn('Rank', regexp_extract(col('Top Ten'), '.', 4))
result= df.where(F.col('Top Ten').rlike("#1"))
result.filter(F.col('Review #')=='1964').show()

WIP
Extract some text from a field
+--------+-----+--------------------+-----+-------+-----+--------+
|Review #|Brand|             Variety|Style|Country|Stars| Top Ten|
+--------+-----+--------------------+-----+-------+-----+--------+
|    1964| MAMA|Instant Noodles C...| Pack|Myanmar|    5|2016 #10|
+--------+-----+--------------------+-----+-------+-----+--------+

